# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. Dumont  ->  A. Dumont  |  ['A. Dumont']
N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']
A. Feldmeier-Krause  ->  A. Feldmeier-Krause  |  ['A. Feldmeier-Krause']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. He  ->  J. He  |  ['J. He']


X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 112 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2307.01252


extracting tarball to tmp_2307.01252...

 done.


A. Dumont  ->  A. Dumont  |  ['A. Dumont']
N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']
A. Feldmeier-Krause  ->  A. Feldmeier-Krause  |  ['A. Feldmeier-Krause']


...

Found 98 bibliographic references in tmp_2307.01277/paper.bbl.
Retrieving document from  https://arxiv.org/e-print/2307.01605


extracting tarball to tmp_2307.01605...

 done.
Retrieving document from  https://arxiv.org/e-print/2307.01724


extracting tarball to tmp_2307.01724...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.01252-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.01252) | **ReveaLLAGN 0: First Look at JWST MIRI data of Sombrero and NGC 1052**  |
|| K. Goold, et al. -- incl., <mark>A. Dumont</mark>, <mark>N. Neumayer</mark>, <mark>A. Feldmeier-Krause</mark> |
|*Appeared on*| *2023-07-06*|
|*Comments*| *Submitted to ApJ*|
|**Abstract**| We present the first results from the Revealing Low-Luminosity Active Galactic Nuclei (ReveaLLAGN) survey, a JWST survey of seven nearby LLAGN. We focus on two observations with the Mid-Infrared Instrument's (MIRI) Medium Resolution Spectrograph (MRS) of the nuclei of NGC 1052 and Sombrero (NGC 4594 / M104). We also compare these data to public JWST data of a higher-luminosity AGN, NGC 7319. JWST clearly resolves the AGN component even in Sombrero, the faintest target in our survey; the AGN components have very red spectra. We find that the emission-line widths in both NGC 1052 and Sombrero increase with increasing ionization potential, with FWHM > 1000 km/s for lines with ionization potential > 50 eV. These lines are also significantly blue-shifted in both LLAGN. The high ionization potential lines in NGC 7319 show neither broad widths or significant blue shifts. Many of the lower ionization potential emission lines in Sombrero show significant blue wings extending > 1000 km/s. These features and the emission-line maps in both galaxies are consistent with outflows along the jet direction. Sombrero has the lowest luminosity high-ionization potential lines ([Ne V] and [O IV]) ever measured in the mid-IR, but the relative strengths of these lines are consistent with higher luminosity AGN. On the other hand, the [Ne V] emission is much weaker relative to the [Ne III}] and [Ne II] lines of higher-luminosity AGN. These initial results show the great promise that JWST holds for identifying and studying the physical nature of LLAGN. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.01259-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.01259) | **Mapping the imprints of stellar and AGN feedback in the circumgalactic  medium with X-ray microcalorimeters**  |
|| G. Schellenberger, et al. -- incl., <mark>A. Pillepich</mark> |
|*Appeared on*| *2023-07-06*|
|*Comments*| *38 pages, 18 figures, submitted to ApJ*|
|**Abstract**| The Astro2020 Decadal Survey has identified the mapping of the circumgalactic medium (CGM, gaseous plasma around galaxies) as a key objective. We explore the prospects for characterizing the CGM in and around nearby galaxy halos with future large grasp X-ray microcalorimeters. We create realistic mock observations from hydrodynamical simulations (EAGLE, IllustrisTNG, and Simba) that demonstrate a wide range of potential measurements, which will address the open questions in galaxy formation and evolution. By including all background and foreground components in our mock observations, we show why it is impossible to perform these measurements with current instruments, such as X-ray CCDs, and only microcalorimeters will allow us to distinguish the faint CGM emission from the bright Milky Way (MW) foreground emission lines. We find that individual halos of MW mass can, on average, be traced out to large radii, around R500, and for larger galaxies even out to R200, using the OVII, OVIII, or FeXVII emission lines. Furthermore, we show that emission line ratios for individual halos can reveal the radial temperature structure. Substructure measurements show that it will be possible to relate azimuthal variations to the feedback mode of the galaxy. We demonstrate the ability to construct temperature, velocity, and abundance ratio maps from spectral fitting for individual galaxy halos, which reveal rotation features, AGN outbursts, and enrichment. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.01269-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.01269) | **Properties of the Line-of-Sight Velocity Field in the Hot and X-ray  Emitting Circumgalactic Medium of Nearby Simulated Disk Galaxies**  |
|| J. A. ZuHone, et al. -- incl., <mark>A. Pillepich</mark> |
|*Appeared on*| *2023-07-06*|
|*Comments*| *41 pages, 29 figures, submitted to ApJ*|
|**Abstract**| The hot, X-ray-emitting phase of the circumgalactic medium in galaxies is believed to be the reservoir of baryons from which gas flows onto the central galaxy and into which feedback from AGN and stars inject mass, momentum, energy, and metals. These effects shape the velocity fields of the hot gas, which can be observed by X-ray IFUs via the Doppler shifting and broadening of emission lines. In this work, we analyze the gas kinematics of the hot circumgalactic medium of Milky Way-mass disk galaxies from the TNG50 simulation with synthetic observations to determine how future instruments can probe this velocity structure. We find that the hot phase is often characterized by outflows outward from the disk driven by feedback processes, radial inflows near the galactic plane, and rotation, though in other cases the velocity field is more disorganized and turbulent. With a spectral resolution of $\sim$1 eV, fast and hot outflows ($\sim$200-500 km s$^{-1}$) can be measured, depending on the orientation of the galaxy on the sky. The rotation velocity of the hot phase ($\sim$100-200 km s$^{-1}$) can be measured using line shifts in edge-on galaxies, and is slower than that of colder gas phases but similar to stellar rotation velocities. By contrast, the slow inflows ($\sim$50-100 km s$^{-1}$) are difficult to measure in projection with these other components. We find that the velocity measured is sensitive to which emission lines are used. Measuring these flows will help constrain theories of how the gas in these galaxies forms and evolves. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.01277-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.01277) | **X-ray metal line emission from the hot circumgalactic medium: probing  the effects of supermassive black hole feedback**  |
|| N. Truong, et al. -- incl., <mark>A. Pillepich</mark> |
|*Appeared on*| *2023-07-06*|
|*Comments*| *21 pages, 15 figures. Submitted to MNRAS and received a positive referee report*|
|**Abstract**| We derive predictions from state-of-the-art cosmological galaxy simulations for the spatial distribution of the hot circumgalactic medium (CGM, ${\rm [0.1-1]R_{200c}}$) through its emission lines in the X-ray soft band ($[0.3-1.3]$ keV). In particular, we compare IllustrisTNG, EAGLE, and SIMBA and focus on galaxies with stellar mass $10^{10-11.6}\, \MSUN$ at $z=0$. The three simulation models return significantly different surface brightness radial profiles of prominent emission lines from ionized metals such as OVII(f), OVIII, and FeXVII as a function of galaxy mass. Likewise, the three simulations predict varying azimuthal distributions of line emission with respect to the galactic stellar planes, with IllustrisTNG predicting the strongest angular modulation of CGM physical properties at radial range ${\gtrsim0.3-0.5\,R_{200c}}$. This anisotropic signal is more prominent for higher-energy lines, where it can manifest as X-ray eROSITA-like bubbles. Despite different models of stellar and supermassive black hole (SMBH) feedback, the three simulations consistently predict a dichotomy between star-forming and quiescent galaxies at the Milky-Way and Andromeda mass range, where the former are X-ray brighter than the latter. This is a signature of SMBH-driven outflows, which are responsible for quenching star formation. Finally, we explore the prospect of testing these predictions with a microcalorimeter-based X-ray mission concept with a large field-of-view. Such a mission would probe the extended hot CGM via soft X-ray line emission, determine the physical properties of the CGM, including temperature, from the measurement of line ratios, and provide critical constraints on the efficiency and impact of SMBH feedback on the CGM. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.01605-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.01605) | **Cosmology with fast radio bursts in the era of SKA**  |
|| J.-G. Zhang, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2023-07-06*|
|*Comments*| *16 pages, 6 figures*|
|**Abstract**| We present a forecast of the cosmological parameter estimation using fast radio bursts (FRBs) from the upcoming Square Kilometre Array (SKA), focusing on the issues of dark energy, the Hubble constant, and baryon density. We simulate $10^5$ and $10^6$ localized FRBs from a 10-year SKA observation, and find that: (i) using $10^6$ FRB data alone can tightly constrain dark-energy equation of state parameters better than CMB+BAO+SN, providing a single cosmological probe to explore dark energy; (ii) combining the FRB data with gravitational wave standard siren data from 10-year observation with the Einstein Telescope, the Hubble constant can be constrained to a sub-percent level, serving as a powerful low-redshift probe; (iii) using $10^6$ FRB data can constrain the baryon density $\Omega_{\rm b}h$ to a precision of $\sim 0.1\%$. Our results indicate that SKA-era FRBs will provide precise cosmological measurements to shed light on both dark energy and the missing baryon problem, and help resolve the Hubble tension. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.01724-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.01724) | **Calibration of the in-orbit center-of-mass of TaiJi-1**  |
|| X. Wei, et al. -- incl., <mark>J. He</mark> |
|*Appeared on*| *2023-07-06*|
|*Comments*| *8 pages, 9 figures*|
|**Abstract**| Taiji program is a space mission aiming to detect gravitational waves in the low frequency band. Taiji-1 is the first technology demonstration satellite of the Taiji Program in Space, with the gravitational reference sensor (GRS) serving as one of its key scientific payloads. For accurate accelerometer measurements, the test-mass center of the GRS must be positioned precisely at the center of gravity of the satellite to avoid measurement disturbances caused by angular acceleration and gradient. Due to installation and measurement errors, fuel consumption during in-flight phase, and other factors, the offset between the test-mass center and the center-of-mass (COM) of the satellite can be significant, degrading the measurement accuracy of the accelerometer. Therefore, the offset needs to be estimated and controlled within the required range by the center-of-mass adjustment mechanism during the satellite's lifetime. In this paper, we present a novel method, the Extended Kalman Filter combined with Rauch-Tung-Striebel Smoother, to estimate the offset, while utilizing the chi-square test to eliminate outliers. Additionally, the nonlinear Least Squares estimation algorithm is employed as a crosscheck to estimate the offset of COM. The two methods are shown to give consistent results, with the offset estimated to be $dx \approx $$-$$0.19$ mm, $dy \approx 0.64$ mm, and $dz \approx $$-$$0.82$ mm. The results indicate a significant improvement on the noise level of GRS after the COM calibration, which will be of great help for the future Taiji program. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2307.01252.md
    + _build/html/tmp_2307.01252/./IP_triple_plot_errbar.png
    + _build/html/tmp_2307.01252/./compare_spitzer.png
    + _build/html/tmp_2307.01252/./compare_spec.png
exported in  _build/html/2307.01259.md
    + _build/html/tmp_2307.01259/./osullivan_maps_358608_prediction.png
    + _build/html/tmp_2307.01259/./osullivan_maps_467415_prediction.png
    + _build/html/tmp_2307.01259/./Sample_Properties.png
exported in  _build/html/2307.01269.md
    + _build/html/tmp_2307.01269/./subhalo_372754_edge_maps.png
    + _build/html/tmp_2307.01269/./subhalo_372754_face_maps.png
    + _build/html/tmp_2307.01269/./372754_profiles.png
    + _build/html/tmp_2307.01269/./subhalo_572328_edge_maps.png
    + _build/html/tmp_2307.01269/./subhalo_572328_face_maps.png
exported in  _build/html/2307.01277.md
    + _build/html/tmp_2307.01277/./figures/Fig_Profile_Dens_Profiles_All.png
    + _build/html/tmp_2307.01277/./figures/Fig_Profile_Tmw_Profiles_All_New.png
    + _

## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\oltext}[1]{\textcolor{olive}{#1}}$
$\newcommand{\rtext}[1]{\textcolor{red}{#1}}$
$\newcommand{\nefive}{[\ion{Ne}{5}] \lambda14.32 \mum}$
$\newcommand{\ofour}{[\ion{O}{4}] \lambda25.91 \mum}$
$\newcommand{\nethree}{[\ion{Ne}{3}] \lambda15.56 \mum}$
$\newcommand{\netwo}{[\ion{Ne}{2}] \lambda12.81 \mum}$
$\newcommand{\fetwo}{[\ion{Fe}{2}] \lambda5.34 \mum}$</div>



<div id="title">

# ReveaLLAGN 0: First Look at JWST MIRI data of Sombrero and NGC 1052

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2307.01252-b31b1b.svg)](https://arxiv.org/abs/2307.01252)<mark>Appeared on: 2023-07-06</mark> -  _Submitted to ApJ_

</div>
<div id="authors">

K. Goold, et al. -- incl., <mark>A. Dumont</mark>, <mark>N. Neumayer</mark>, <mark>A. Feldmeier-Krause</mark>

</div>
<div id="abstract">

**Abstract:** We present the first results from the Revealing Low-Luminosity Active Galactic Nuclei (ReveaLLAGN) survey, a JWST survey of seven nearby LLAGN.  We focus on two observations with the Mid-Infrared Instrument's (MIRI) Medium Resolution Spectrograph (MRS) of the nuclei of NGC 1052 and Sombrero (NGC 4594 / M104).  We also compare these data to public JWST data of a higher-luminosity AGN, NGC 7319.   JWST clearly resolves the AGN component even in Sombrero, the faintest target in our survey; the AGN components have very red spectra.   We find that the emission-line widths in both NGC 1052 and Sombrero increase with increasing ionization potential, with FWHM $>$ 1000 km $ $ s $^{-1}$ for lines with ionization potential $\gtrsim$ 50 eV.  These lines are also significantly blue-shifted in both LLAGN.  The high ionization potential lines in NGC 7319 show neither broad widths or significant blue shifts.  Many of the lower ionization potential emission lines in Sombrero show significant blue wings extending $>$ 1000 km $ $ s $^{-1}$ .  These features and the emission-line maps in both galaxies are consistent with outflows along the jet direction.  Sombrero has the lowest luminosity high-ionization potential lines ( [ $\ion{Ne}{5}$ ] and [ $\ion{O}{4}$ ] ) ever measured in the mid-IR, but the relative strengths of these lines are consistent with higher luminosity AGN.  On the other hand, the [ $\ion{Ne}{5}$ ] emission is much weaker relative to the [ $\ion{Ne}{3}$ ] and [ $\ion{Ne}{2}$ ] lines of higher-luminosity AGN.  These initial results show the great promise that JWST holds for identifying and studying the physical nature of LLAGN.

</div>

<div id="div_fig1">

<img src="tmp_2307.01252/./IP_triple_plot_errbar.png" alt="Fig1" width="100%"/>

**Figure 1. -** Emission-line trends with ionization potential.  Emission features are listed along the x-axis ordered by their IP.  *Top -- Luminosity vs IP*. Emission-line luminosities scale with the Eddington ratio of sources. NGC 7319 has the highest Eddington ratio and the most luminous emission lines, followed by NGC 1052, and then Sombrero. The luminosities have a median fractional error of 15\%.
    *Middle -- FWHM$_{line*$ vs IP}. The FWHM$_{\rm line}$ of emission features increases with IP in Sombrero and NGC 1052 while the FWHM$_{\rm line}$ of NGC 7319 emission features stays relatively constant with IP. FWHM$_{\rm line}$ in km s$^{-1}$ is shown on the y-axis with a median error of 30 km s$^{-1}$. Red and blue dashed lines represent the central stellar velocity dispersion measurements from \citet{Ho2009b} translated to a FWHM.
    *Bottom -- Peak Velocity vs IP*. Peak velocity of emission lines trend increasingly blue-shifted with increasing ionization potential in Sombrero and NGC 1052. The y-axis shows the peak velocity of the best fit Guassian model with a median error of 30 km s$^{-1}$.  (*fig:lum-fwhm-vel*)

</div>
<div id="div_fig2">

<img src="tmp_2307.01252/./compare_spitzer.png" alt="Fig3" width="100%"/>

**Figure 3. -** JWST enables us to separate LLAGN spectra from their host galaxy.
 Comparison of the aperture corrected nuclear extracted spectrum in Sombrero (red line; same as Figure \ref{fig:nucspec}), to the integrated MIRI/MRS spectrum (gray line; FoV: 6$\farcs$6$\times$7$\farcs$7),and the Spitzer LR spectrum  (magenta line; FoV: 27$\farcs$7$\times$51$\farcs$8).  The black line shows the best fit high-spatial resolution power-law fit to Sombrero from \citet{Fern2023} this is fit to the black points, which are photometry from Gemini \citep{Asmus2014} and VLT \citep{Fern2023} as well as  sub-arcsecond data at shorter wavelengths; both the data and fit are in good agreement with our nuclear spectrum. We show the integrated spectrum only out to 20 $\mu$m as the poorly constrained MIRI channel 4 background levels significantly impact the integrated spectrum measurements at redder wavelengths.  (*fig:spitzercomp*)

</div>
<div id="div_fig3">

<img src="tmp_2307.01252/./compare_spec.png" alt="Fig6" width="100%"/>

**Figure 6. -** The first extracted nuclear spectra of ReveaLLAGN targets: Sombrero (red, bottom spectrum) and NGC 1052 (blue, top spectrum). NGC 7319 (black, middle spectrum) is a more distant Seyfert 2 AGN and is included to compare our low-luminosity sample to another spectrum taken with JWST MIRI/MRS.  Spectra are extracted from a $\sim$1 FWHM radius aperture (see Section \ref{sec:nuclear_extraction}) and are aperture corrected using point source observations.  A subset of strong emission lines are labeled.  Also apparent in the spectra at $\sim$10 microns are broad Silicate absorption features (in NGC 7319) and emission features (in Sombrero and NGC 1052), and faint polycyclic aromatic hydrocarbon (PAH) emission at 11.3 $\mu$m in Sombrero.  (*fig:nucspec*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2307.01252"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\kms}{km s^{-1}}$</div>



<div id="title">

# Mapping the imprints of stellar and AGN feedback in the circumgalactic medium with X-ray microcalorimeters

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2307.01259-b31b1b.svg)](https://arxiv.org/abs/2307.01259)<mark>Appeared on: 2023-07-06</mark> -  _38 pages, 18 figures, submitted to ApJ_

</div>
<div id="authors">

G. Schellenberger, et al. -- incl., <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** The Astro2020 Decadal Survey has identified the mapping of the circumgalactic medium (CGM, gaseous plasma around galaxies) as a key objective.We explore the prospects for characterizing the CGM in and around nearby galaxy halos with future large grasp X-ray microcalorimeters.We create realistic mock observations from hydrodynamical simulations (EAGLE, IllustrisTNG, and Simba) that demonstrate a wide range of potential measurements, which will address the open questions in galaxy formation and evolution.By including all background and foreground components in our mock observations, we show why it is impossible to perform these measurements with current instruments, such as X-ray CCDs, and only microcalorimeters will allow us to distinguish the faint CGM emission from the bright Milky Way (MW) foreground emission lines.We find that individual halos of MW mass can, on average, be traced out to large radii, around R $_{500}$ , and for larger galaxies even out to R $_{200}$ , using the O VII, O VIII, or Fe XVII emission lines.Furthermore, we show that emission line ratios for individual halos can reveal the radial temperature structure. Substructure measurements show that it will be possible to relate azimuthal variations to the feedback mode of the galaxy.We demonstrate the ability to construct temperature, velocity, and abundance ratio maps from spectral fitting for individual galaxy halos, which reveal rotation features, AGN outbursts, and enrichment.

</div>

<div id="div_fig1">

<img src="tmp_2307.01259/./osullivan_maps_358608_prediction.png" alt="Fig15" width="100%"/>

**Figure 15. -** Spectral map of galaxy 358608 from TNG50 (for details see text), showing observed O VIII surface brightness (a), and the predicted optical r-band signal (b), the simulated X-ray brightness (c), the observed temperature in keV (c, and error map d) from a simultaneous fit to O VII, O VIII, and Fe XVII lines), the emission weighted temperature from the simulation (f),
    the observed average line velocity shift in $\si{km s^{-1}}$(g, and error map h), the predicted, emission weighted LOS velocity in the simulation (i), the observed O/Fe abundance ratio (g, and error map h), and the predicted O/Fe brightness ratio in the simulation (l).  (*fig:specmap358608*)

</div>
<div id="div_fig2">

<img src="tmp_2307.01259/./osullivan_maps_467415_prediction.png" alt="Fig16" width="100%"/>

**Figure 16. -** As for Fig. \ref{fig:specmap358608}, but for the lower mass galaxy from TNG50 (ID 467415, details in the text). (*fig:specmap467415*)

</div>
<div id="div_fig3">

<img src="tmp_2307.01259/./Sample_Properties.png" alt="Fig7" width="100%"/>

**Figure 7. -** Properties of the galaxies in the three samples (low/medium/high mass) for each of the three simulations (TNG100 in blue, EAGLE in orange, Simba in green). The x-axis of each of the three panels is the halo mass M$_{200}$ in log$_{10}$. The left panel shows the stellar mass dependence M$_\star$, similar to what is shown in \ref{fig:stellar_mass}. We also highlight the location of the Milky Way (purple star), and M31 (red star). The middle panel illustrates the specific star formation rate, sSFR in units of $\si{yr^{-1}}$, clearly decreasing with increasing halo mass. The right panel shows the SMBH mass dependence, clearly indicating a different trend between the simulations, where EAGLE produces lower mass black holes than TNG100. (*fig:samples*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2307.01259"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Properties of the Line-of-Sight Velocity Field in the Hot and X-ray Emitting Circumgalactic Medium of Nearby Simulated Disk Galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2307.01269-b31b1b.svg)](https://arxiv.org/abs/2307.01269)<mark>Appeared on: 2023-07-06</mark> -  _41 pages, 29 figures, submitted to ApJ_

</div>
<div id="authors">

J. A. ZuHone, et al. -- incl., <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** The hot, X-ray-emitting phase of the circumgalactic medium in galaxies is believed to be the reservoir of baryons from which gas flows onto the central galaxy and into which feedback from AGN and stars inject mass, momentum, energy, and metals. These effects shape the velocity fields of the hot gas, which can be observed by X-ray IFUs via the Doppler shifting and broadening of emission lines. In this work, we analyze the gas kinematics of the hot circumgalactic medium of Milky Way-mass disk galaxies from the TNG50 simulation, and produce synthetic observations to determine how future instruments can probe this velocity structure. We find that the hot phase is often characterized by outflows outward from the disk driven by feedback processes, radial inflows near the galactic plane, and rotation, though in other cases the velocity field is more disorganized and turbulent. With a spectral resolution of $\sim$ 1 eV, fast and hot outflows ( $\sim$ 200-500 km s $^{-1}$ ) can be measured using both line shifts and widths, depending on the orientation of the galaxy on the sky. The rotation velocity of the hot phase ( $\sim$ 100-200 km s $^{-1}$ ) can be measured using line shifts in edge-on galaxies, and is slower than that of colder gas phases but similar to stellar rotation velocities. By contrast, the slow inflows ( $\sim$ 50-100 km s $^{-1}$ ) are difficult to measure in projection with these other components. We find that the velocity measured is sensitive to which emission lines are used. Measuring these flows will help constrain theories of how the gas in these galaxies forms and evolves.

</div>

<div id="div_fig1">

<img src="tmp_2307.01269/./subhalo_372754_edge_maps.png" alt="Fig1.1" width="50%"/><img src="tmp_2307.01269/./subhalo_372754_face_maps.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** Projections of various quantities from galaxy 1, viewed edge-on (upper panels) and face-on (lower panels) with respect to the galactic plane. For each set, top row from left: Stellar mass density, X-ray SB in the 0.5-1.0 keV band, emission-weighted gas temperature. Bottom row from left: emission-weighted gas mean velocity, emission-weighted gas velocity dispersion. Each panel is 32' on a side, or $\sim$407 kpc for the given redshift and cosmology. Hot outflows are evident in SB and temperature, which are associated with fast mean velocities and velocity dispersions near the center. The hot gas is also rotating, as seen in the edge-on projection. (*fig:gal1_proj*)

</div>
<div id="div_fig2">

<img src="tmp_2307.01269/./372754_profiles.png" alt="Fig9" width="100%"/>

**Figure 9. -** Azimuthally and height-averaged mass-weighted and emission-weighted radial profiles of the gas and stellar velocity for galaxy 1. The top panels show profiles of the $R$-component of the velocity, the middle panels show profiles of the $\phi$-component, and the bottom panels show the $z$-component. Left panels show the mean velocity for each galaxy and right panels show the velocity dispersion. The same patterns of outflows, inflows, and rotation are seen as in the 2D profile in Figure \ref{fig:gal1_phase}. The emission-weighted velocity profiles are similar to the mass-weighted profiles, but higher-energy lines (Fe XVII and Ne IX) track hotter and faster phases of gas. (*fig:gal1_profiles*)

</div>
<div id="div_fig3">

<img src="tmp_2307.01269/./subhalo_572328_edge_maps.png" alt="Fig3.1" width="50%"/><img src="tmp_2307.01269/./subhalo_572328_face_maps.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** Projections of various quantities from galaxy 6, viewed edge-on (upper panels) and face-on (lower panels) with respect to the plane of the galactic disk. Panel descriptions are the same as in Figure \ref{fig:gal1_proj}. Each panel is 32' on a side, or $\sim$407 kpc for the given redshift and cosmology. Unlike galaxies 1 and 2, this galaxy has no obvious pattern of rotation in the velocity map, and the fast outflow appears to not be aligned with the spin axis of the disk. (*fig:gal6_proj*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2307.01269"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\MSUN}{\rm{M}_{\sun}}$
$\newcommand{\RVIR}{{R}_{\rm 200c}}$
$\newcommand{\MHOST}{{M}_{\rm 200c, host}}$
$\newcommand{\MSTARS}{{M}_{*}}$
$\newcommand{\ap}[1]{{\color{magenta} #1}}$
$\newcommand{\nt}[1]{{\color{blue} #1}}$
$\newcommand{\arraystretch}{1.1}$</div>



<div id="title">

# X-ray metal line emission from the hot circumgalactic medium: probing the effects of supermassive black hole feedback

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2307.01277-b31b1b.svg)](https://arxiv.org/abs/2307.01277)<mark>Appeared on: 2023-07-06</mark> -  _21 pages, 15 figures. Submitted to MNRAS and received a positive referee report_

</div>
<div id="authors">

N. Truong, et al. -- incl., <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** We derive predictions from state-of-the-art cosmological galaxy simulations for the spatial distribution of the hot circumgalactic medium (CGM, ${\rm[0.1-1]R_{200c}}$ ) through its emission lines in the X-ray soft band ( $[0.3-1.3]$ keV). In particular, we compare IllustrisTNG, EAGLE, and SIMBA and focus on galaxies with stellar mass $10^{10-11.6}  \MSUN$ at $z=0$ . The three simulation models return significantly different surface brightness radial profiles of prominent emission lines from ionized metals such as OVII(f), OVIII, and FeXVII as a function of galaxy mass. Likewise, the three simulations predict varying azimuthal distributions of line emission with respect to the galactic stellar planes, with IllustrisTNG predicting the strongest angular modulation of CGM physical properties at radial range ${\gtrsim0.3-0.5 R_{200c}}$ . This anisotropic signal is more prominent for higher-energy lines, where it can manifest as X-ray eROSITA-like bubbles. Despite different models of stellar and supermassive black hole (SMBH) feedback, the three simulations consistently predict a dichotomy between star-forming and quiescent galaxies at the Milky-Way and Andromeda mass range, where the former are X-ray brighter than the latter. This is a signature of SMBH-driven outflows, which are responsible for quenching star formation. Finally, we explore the prospect of testing these predictions with a microcalorimeter-based X-ray mission concept with a large field-of-view. Such a mission would probe the extended hot CGM via soft X-ray line emission, determine the physical properties of the CGM, including temperature, from the measurement of line ratios, and provide critical constraints on the efficiency and impact of SMBH feedback on the CGM.

</div>

<div id="div_fig1">

<img src="tmp_2307.01277/./figures/Fig_Profile_Dens_Profiles_All.png" alt="Fig6.1" width="33%"/><img src="tmp_2307.01277/./figures/Fig_Profile_Tmw_Profiles_All_New.png" alt="Fig6.2" width="33%"/><img src="tmp_2307.01277/./figures/Fig_Profile_Zmw_Profiles_All_New.png" alt="Fig6.3" width="33%"/>

**Figure 6. -** 2D projected radial profiles of the CGM thermodynamical and metal content across galaxy stellar mass ranges (different columns) in TNG100, EAGLE, and SIMBA. From _ top_ to _ bottom_ panels we show the gas column density (${\rm \Sigma_{gas}}$), gas mass-weighted temperature (${\rm T_{mw}}$), and gas mass-weighted metallicity (${\rm Z_{mw}}$). The solid lines represent the population median, whereas the shaded areas show the 16th and 84th percentiles, i.e. quantify the galaxy-to-galaxy variation. (*fig:3*)

</div>
<div id="div_fig2">

<img src="tmp_2307.01277/./figures/Fig_Profile_CV_Profiles_All_New.png" alt="Fig7.1" width="25%"/><img src="tmp_2307.01277/./figures/Fig_Profile_OVIIf_Profiles_All_New.png" alt="Fig7.2" width="25%"/><img src="tmp_2307.01277/./figures/Fig_Profile_OVIII_Profiles_All_New.png" alt="Fig7.3" width="25%"/><img src="tmp_2307.01277/./figures/Fig_Profile_FeXVII_Profiles_All_New.png" alt="Fig7.4" width="25%"/>

**Figure 7. -** Similar to Fig. \ref{fig:3} but for the surface brightness profile of a selected sample of emitting lines in the three simulations. From _ top_ to _ bottom_ we show profiles of: CVI, OVIIf, OVIII, and FeXVII (intrinsic emission from the diffuse gas, as per Section \ref{sec:computation}). The horizontal grey lines represent the 20 per cent level of the LEM background plus foreground (see text for more details), and their horizontal extension specifies the area covered by LEM field of view at $z=0.01$(with radius given in $\RVIR$). For context, the median $\RVIR$ of the depicted TNG galaxies reads 188, 245, 354, 537 kpc, from left to right. (*fig:4*)

</div>
<div id="div_fig3">

<img src="tmp_2307.01277/./figures/Fig_Profile_OVIIf_Profiles_Sf_vs_Quiescent.png" alt="Fig8.1" width="33%"/><img src="tmp_2307.01277/./figures/Fig_Profile_OVIII_Profiles_Sf_vs_Quiescent.png" alt="Fig8.2" width="33%"/><img src="tmp_2307.01277/./figures/Fig_Profile_FeXVII_Profiles_Sf_vs_Quiescent.png" alt="Fig8.3" width="33%"/>

**Figure 8. -** Diversity between star-forming and quiescent galaxies at the MW-mass range (${$\MSTARS$=10^{10.7\pm0.1}  $\MSUN$}$) according to TNG100, EAGLE and SIMBA at $z=0$. From _ top_ to _ bottom_, we show the intrinsic surface brightness profiles of OVIIf, OVIII, and FeXVII line emission, respectively. Galaxies are split based on their star-formation status: star-forming (blue) and quiescent (red) galaxies depending on their ${\rm sSFR}$ with respect to quenched threshold ${\rm< 10^{-11}yr^{-1}}$. The solid curves denote the median profile of each subgroup, and the shaded areas represent the 16th-84th percentile envelope. Despite the vastly different implementations of SMBH feedback, all considered galaxy formation models predict that, at this mass range, the (inner) CGM of star-forming galaxies is brighter than that around quiescent galaxies, in each of the specific soft X-ray emission lines we consider. This is because, according to the models, quiescent galaxies exhibit overall less dense, less enriched, but hotter halo gas in comparison to star-forming galaxies (see Fig. \ref{fig:cgm_sf_q}). (*fig:6b*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2307.01277"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

226  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
